In [1]:
import openai
import json
import yaml
import boto3

import sys
import os

import wandb

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from src.gpt_wrappers import ChatGPTWithMemory
from src.quality_checks import *
from src.xml_utils import *

In [2]:
# Load config
with open('../config.yml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

openai.api_key = config['OPENAI_KEY']

s3 = boto3.resource(
    's3',
    aws_access_key_id=config['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=config['AWS_SECRET_ACCESS_KEY']
)

run = wandb.init(
    project=config['WANDB_PROJECT'], 
    entity=config['WANDB_ENTITY']
)

results_table = wandb.Table(
    columns=[
                "system_message", 
                "user_message",
                "assistant_message",
                "question",
                "answer",
                "number_of_questions",
    ]
)

wandb: Currently logged in as: rachnogstyle. Use `wandb login --relogin` to force relogin


In [3]:
system_prompt = \
"""
    You are a teacher, educator and practitioner. 
    Your goal is to prepare questions for the students that are engaging, motivating and challenging.
    You can take texts from the internet, books, articles, etc. and use them to create questions.
    For each piece of text you can generate set of questions with indicated difficulty level and answers.
    You can generate tests with single or multiple choice questions.
    You can generate open-ended questions. You can generate debate topics.
    You can generate provocative topics for the essays that students can write.
    You can generate ideas for practical group projects or even business ideas based on the text.
    IMPORTANT: design questions in such a way, that ChatGPT and similar AI systems cannot answer them.
"""

example_question = ""
example_answer = ""

question = \
    """
        Here is the input text:

        23 Wall Street, also known as the J.P. Morgan Building, is an office building in the Financial District of Manhattan in New York City, at the southeast corner of Wall Street and Broad Street. The four-story building, designed by Trowbridge & Livingston in the Neoclassical style, was constructed between 1913 and 1914. When it was damaged during the Wall Street bombing in 1920, J.P. Morgan & Co. refused to make repairs to defy the bombers. The building was the firm's headquarters until 1989, when the company moved to 60 Wall Street. During the 2000s, there were plans to convert 23 Wall Street into condominiums. The building was sold in 2008 to interests associated with the billionaire industrialist Sam Pa. It has mostly remained empty, although it has been used for events. The building is a New York City designated landmark and is listed on the National Register of Historic Places (NRHP); it is also a contributing property to the NRHP-listed Wall Street Historic District. 

        Based on the text, generate please:
        - 1 debate topic
        - 1 social project idea
        - 1 business idea

        Separate each thing with a new line (\n\n) so I can parse it later.
    """

In [4]:
chatgpt = ChatGPTWithMemory(system_prompt)
chatgpt.initialize_with_question_answer(example_question, example_answer)
answer = chatgpt.generate(question)
# save_xml_string_to_file(answer, '../data/text.xml')
quality_check = TextToQuestionsQualityCheck(answer)

results_table.add_data(
    *[
        system_prompt, example_question, example_answer, question, answer, quality_check.check_number_of_questions()
    ]
)

In [7]:
print(answer), quality_check.check_number_of_questions()

- Debate topic: Should historic buildings like 23 Wall Street be converted into condominiums for modern use or preserved as landmarks for future generations?
- Social project idea: Launch a program that works with local schools to bring students on guided tours of historic buildings like 23 Wall Street, along with workshops about the importance of protecting and preserving cultural heritage.
- Business idea: Create a startup that uses augmented reality (AR) technology to offer virtual tours of historic sites like 23 Wall Street, allowing people from around the world to experience and appreciate these landmarks without physical travel.


(None, 3)

In [6]:
run.log({"results_table": results_table})